In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [2]:
data_ex = os.listdir('Species/Training_Set/Anthias anthias')
print(data_ex)

['anthias_anthias-0.jpg', 'anthias_anthias-1.jpg', 'anthias_anthias-10.jpg', 'anthias_anthias-100.jpg', 'anthias_anthias-1000.jpg', 'anthias_anthias-1001.jpg', 'anthias_anthias-1002.jpg', 'anthias_anthias-1003.jpg', 'anthias_anthias-1004.jpg', 'anthias_anthias-1005.jpg', 'anthias_anthias-1006.jpg', 'anthias_anthias-1007.jpg', 'anthias_anthias-1008.jpg', 'anthias_anthias-1009.jpg', 'anthias_anthias-101.jpg', 'anthias_anthias-1010.jpg', 'anthias_anthias-1011.jpg', 'anthias_anthias-1012.jpg', 'anthias_anthias-1013.jpg', 'anthias_anthias-1014.jpg', 'anthias_anthias-1015.jpg', 'anthias_anthias-1016.jpg', 'anthias_anthias-1017.jpg', 'anthias_anthias-1018.jpg', 'anthias_anthias-1019.jpg', 'anthias_anthias-102.jpg', 'anthias_anthias-1020.jpg', 'anthias_anthias-1021.jpg', 'anthias_anthias-1022.jpg', 'anthias_anthias-1023.jpg', 'anthias_anthias-1024.jpg', 'anthias_anthias-1025.jpg', 'anthias_anthias-1026.jpg', 'anthias_anthias-1027.jpg', 'anthias_anthias-1028.jpg', 'anthias_anthias-1029.jpg', 'a

In [3]:
train_dir = os.listdir('Species/Training_Set/')
test_dir = os.listdir('Species/Test_Set/')

In [4]:
len(train_dir)

20

In [5]:
len(test_dir)

20

In [6]:
targetnames = ['Anthias anthias','Atherinomorus lacunosus','Belone belone','Boops boops','Chlorophthalmus agassizi',
               'Coris julis','Dasyatis centroura','Epinephelus caninus','Gobius niger','Mugil cephalus','Phycis phycis',
              'Polyprion americanus','Pseudocaranx dentex','Rhinobatos cemiculus','Scomber japonicus','Solea solea',
               'Squalus acanthias','Tetrapturus belone','Trachinus draco','Trigloporus lastoviza']

In [ ]:
# Augmenting images and storing them in temporary directories 
for img_class in targetnames:

    #creating temporary directories
    # creating a base directory
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)
    # creating a subdirectory inside the base directory for images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    img_list = os.listdir('Species/Training_Set/' + img_class)

    # Copy images from the class train dir to the img_dir 
    for file_name in img_list:

        # path of source image in training directory
        source = os.path.join('Species/Training_Set/' + img_class, file_name)

        # creating a target directory to send images 
        target = os.path.join(img_dir, file_name)

        # copying the image from the source to target file
        shutil.copyfile(source, target)

    # Temporary augumented dataset directory.
    source_path = aug_dir

    # Augmented images will be saved to training directory
    save_path = 'Species/Filtered_Noise/' + img_class

    # Creating Image Data Generator to augment images
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(

        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'

    )

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(source_path,save_to_dir=save_path,save_format='jpg',target_size=(224, 224),batch_size=batch_size)

    # Generate the augmented images
    aug_images = 8000 

    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((aug_images - num_files) / batch_size))

    # creating 8000 augmented images per class
    for i in range(0, num_batches):
        images, labels = next(aug_datagen)

    # delete temporary directory 
    shutil.rmtree('aug_dir')

Found 1700 images belonging to 1 classes.
Found 1700 images belonging to 1 classes.


C:\Users\edwal\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [7]:
train_path = 'Species/Filtered_Noise/'
test_path = 'Species/Test_Set/'
batch_size=16

In [8]:
datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)

In [9]:
image_size = 380
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 137600 images belonging to 20 classes.

Test Batches: 
Found 6000 images belonging to 20 classes.


In [10]:
rm = tf.keras.applications.EfficientNetB4(
    input_shape = (380, 380, 3), include_top = False, weights = 'imagenet'
)
for layer in rm.layers:
    layer.trainable = False
    x = rm.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(5, activation="sigmoid")(x)
model = Model(inputs = rm.input, outputs = predictions)
model.summary()

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [ ]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.01,epsilon=0.1)
model.compile(optimizer=opt1,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
class_weights = {   
                    0: 1.0,  # Anthias anthias
                    1: 1.0,  # Atherinomorus Laconusus
                    2: 1.0,  # Belone belone
                    3: 1.0,  # Boops boops
                    4: 1.0,  # Chlorophthalmus agassizi
                    5: 1.0,  # Coris julis
                    6: 1.0,  # Dasyatis centroura
                    7: 1.0,  # Epinephelus caninus
                    8: 1.0,  # Gobius niger
                    9: 1.0,  # Mugil cephalus
                    10: 1.0,  # Phycis phycis
                    11: 1.0,  # Polyprion americanus
                    12: 1.0,  # Pseudocaranx dentex
                    13: 1.0,  # Rhinobatos cemiculus
                    14: 1.0,  # Scomber japonicus
                    15: 1.0,  # Solea solea
                    16: 1.0,  # Squalus acanthias
                    17: 1.0,  # Tetrapturus belone
                    18: 1.0,  # Trachinus draco
                    19: 1.0,  # Trigloporus lastoviza
                }


checkpoint=  ModelCheckpoint(filepath = 'EfficientNetB4.hdf5',monitor='val_accuracy',save_best_only=True,save_weights_only=True)